# Disclaimer

If you are publishing data analysed by this software package please cite: DOI:10.5281/zenodo.1469364

Special thanks go to Duncan Johnstone, Elena Pascal, Paul R. Edwards and Jordi Ferrer-Orri in helping to create this particular analysis File

In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs
import h5py
import os

C:\Users\Gunnar\anaconda3\lib\site-packages\pyUSID\viz\__init__.py:16: FutureWarning: Please use sidpy.viz.plot_utils instead of pyUSID.viz.plot_utils. pyUSID.plot_utils will be removed in a future release of pyUSID
  warn('Please use sidpy.viz.plot_utils instead of pyUSID.viz.plot_utils. '


# Get values from MicroscopeStatus:

In [2]:
with open('MicroscopeStatus.txt', encoding='windows-1252' ) as status :
    for line in status:
        #if 'Field of view'  in line:
         #   calax = float(line[-9:-4] )    #calax= micro meter per pixel
        if 'Horizontal Binning' in line:
            binning = int(line[line.find(':')+1:-1])        #binning = binning status
        if 'Resolution_X' in line:
            nx = int(line[line.find(':')+1:-8])         #nx = pixel in x-direction
        if 'Resolution_Y' in line:
            ny = int(line[line.find(':')+1:-8])         #ny = pixel in y-direction
        if 'Real Magnification' in line:
             FOV = float (line[line.find(':')+1:-2])
        if 'Grating - Groove Density:' in line:
            grating = float (line[line.find(':')+1:-7])
        if 'Camera Model:' in line:
            camera = str ((line[line.find(':')+1:-1]))
        if 'Central wavelength:' in line:
            centrelambda = float (line[line.find(':')+1:-3])

In [9]:
if camera == 'A.920' :
    ch = 1024//binning
    Ebert = 21.2 # Ebert Angle in degree from Horiba website
    lccd = 26.7 # CCD width in mm from Andor Specsheet
    flength = 319.76001 #focal length in mm from horiba specsheet
    gamma = -3.5 #in degree
    lH = flength*np.cos(gamma/180*np.pi)
    hblcentre = flength*np.sin(gamma/180*np.pi)

    alpha = np.arcsin((10**(-6)*grating*centrelambda)/(2*np.cos((Ebert/(2*180))*np.pi)))/np.pi*180-Ebert/2 
    beta = Ebert+alpha

    betamin = beta + gamma - np.arctan((((lccd/ch)  * (ch - ch/2) + hblcentre)/lH))*180/np.pi
    lambdamin = ((np.sin(alpha/180*np.pi)+np.sin(betamin/180*np.pi))*10**6)/grating

    betamax = beta + gamma - np.arctan((((lccd/ch)  * (1 - ch/2) + hblcentre)/lH))*180/np.pi
    lambdamax = ((np.sin(alpha/180*np.pi)+np.sin(betamax/180*np.pi))*10**6)/grating
    
    if grating == 150 :
        corrfactor = 2.73E-04
    elif grating == 600 :
        corrfactor = 6.693659836087227e-05
    elif grating == 1200 :
        corrfactor = 3.7879942917985216e-05
    else :
        print('Something went wrong')
    
elif camera =='A.(IR)490' :
    ch = 512//binning
    Ebert = -11.6348 # Ebert Angle in degree from Horiba website
    lccd = 12.8 # CCD width in mm from Andor Specsheet
    flength = 326.7 #focal length in mm from horiba specsheet
    gamma = -4.8088 #in degree
    lH = flength*np.cos(gamma/180*np.pi)
    hblcentre = flength*np.sin(gamma/180*np.pi)

    alpha = np.arcsin((10**(-6)*grating*centrelambda)/(2*np.cos((Ebert/(2*180))*np.pi)))/np.pi*180-Ebert/2 
    beta = Ebert+alpha

    betamin = beta + gamma - np.arctan((((lccd/ch)  * (ch - ch/2) + hblcentre)/lH))*180/np.pi
    lambdamin = ((np.sin(alpha/180*np.pi)+np.sin(betamin/180*np.pi))*10**6)/grating

    betamax = beta + gamma - np.arctan((((lccd/ch)  * (1 - ch/2) + hblcentre)/lH))*180/np.pi
    lambdamax = ((np.sin(alpha/180*np.pi)+np.sin(betamax/180*np.pi))*10**6)/grating
    
    if grating == 150 :
        corrfactor = 2.73E-04
    else :
        print('Something went wrong')
    
    
else :
    print('Dont know that camera')

    


### Load data into numpy array

In [10]:
filename = 'HYPCard.bin'
with open(filename, 'rb') as f:    
    data = np.fromfile(f, dtype= [('bar', '<i4')], count= ch*nx*ny)
    #data = np.fromfile(f, count= 1024*nx*ny)
    array = np.reshape(data, [ch, nx, ny], order='F')
    
sarray = np.swapaxes(array, 1,2)

suncor = hs.signals.Signal2D(sarray).T
suncor.change_dtype('float')

Swap Axes for proper x-y use

# Define axis 

In [11]:
x = suncor.axes_manager.navigation_axes[0]
y = suncor.axes_manager.navigation_axes[1]

calax = 131072/(FOV*nx)

x.name = 'x'
x.scale = calax * 1000         #changes micrometer to nm, value for the size of 1 pixel
x.units = 'nm'

y.name = 'y'
y.scale = calax * 1000      #changes micrometer to nm, value for the size of 1 pixel
y.units = 'nm'

dx = suncor.axes_manager.signal_axes[0]

dx.name = 'wavelength'
dx.scale = ((lambdamax-lambdamin)/ch)
dx.offset = lambdamin
dx.units = '$nm$'

# Background Correction

In [12]:
 if os.path.exists('BKG1.txt'):
    bkg = np.loadtxt('BKG1.txt', skiprows = 1)
    bkgarray =np.ones((nx,ny, len(bkg)))*bkg
    s = suncor - bkgarray
else:
    s = suncor
    print("Spectra has no Background")

# Correction of the Wavelength Shift along the X-Axis

In [13]:
garray=np.arange((-corrfactor/2) * calax * 1000 * (nx), (corrfactor/2) * calax * 1000 * (nx), corrfactor *calax * 1000) #(Total Variation, Channels, Step)
barray = np.full((nx,ny),garray)
s.shift1D(barray)

# Data Analysis

In [14]:
s.plot()

Plots the map with the LiveSE as the Navigator

In [17]:
Filename = "SE_Scan_"+str(nx)+"_"+str(ny)+"-SE.png"
SE = hs.load(Filename)
s.plot(navigator=SE)

Calculate, plot and save the spatially averaged spectrum (remove the # to save)

In [18]:
sm = s.mean((0,1))
sm.plot()
#sm.save('Meanspectra.hdf5')

Swap axes to navigate on the wavelength axis

In [19]:
im = s.T
im.plot()
#roi1 = hs.roi.SpanROI(left=300, right=400)      #sets a digitalbandfilter
#im_roi1 = roi1.interactive(im, color="red")
#roi2 = hs.roi.SpanROI(left=400, right=500)      #sets another digitalbandfilter
#im_roi2 = roi2.interactive(im, color="blue")
#roi3 = hs.roi.SpanROI(left=500, right=600)      #sets another digitalbandfilter
#im_roi3 = roi3.interactive(im, color="green")

Selected Bandpasses with Image as Navigator

In [ ]:
roi1(im).T.plot( navigator_kwds=dict(colorbar=True,
                             scalebar_color='black',
                             cmap='Reds_r'))

roi2(im).T.plot( navigator_kwds=dict(colorbar=True,
                             scalebar_color='black',
                             cmap='Blues_r'))

roi3(im).T.plot( navigator_kwds=dict(colorbar=True,
                            scalebar_color='black',
                            cmap='Greens_r'))

Take a linescan through map

In [22]:
line = hs.roi.Line2DROI(x1=346.018, y1=1435.37, x2=1747.57, y2=1324.84, linewidth=50)
s.plot()
line.add_widget(s)

In [24]:
trace  = line(s).T.sum(0)
trace.plot()

Take a Linespectrum through the Map

In [20]:
line = hs.roi.Line2DROI(x1=346.018, y1=1435.37, x2=1747.57, y2=1324.84, linewidth=100)
s.plot()
line.add_widget(s)

In [21]:
spectrace  = line(s)
spectrace.plot( navigator_kwds=dict(colorbar=True,
                             scalebar_color='red',
                             cmap='inferno_r',
                             aspect = 0.1))

PCA decomposition - Takes lot of Processing Power/Takes a lot of time

In [170]:
s.decomposition()

In [171]:
s.plot_decomposition_results()

## Gaussian fitting example

In [22]:
m = s.create_model()   #Creates a Model for fitting

In [23]:
g1 = hs.model.components1D.Expression(
expression="height * exp(-(x - x0) ** 2 * 4 * log(2)/ fwhm ** 2)",
name="Groundstate",
position="x0",
height=1,
fwhm=1,
x0=0,
module="numpy")

g2 = hs.model.components1D.Expression(
expression="height * exp(-(x - x0) ** 2 * 4 * log(2)/ fwhm ** 2)",
name="First Excited State",
position="x0",
height=1,
fwhm=1,
x0=0,
module="numpy")

g3 = hs.model.components1D.Expression(
expression="height * exp(-(x - x0) ** 2 * 4 * log(2)/ fwhm ** 2)",
name="Second excited State",
position="x0",
height=1,
fwhm=1,
x0=0,
module="numpy")

bkg = hs.model.components1D.Offset()   # sets background

In [24]:
m.extend([g1,g2, g3, bkg])                    #adds fits and bkg to the model

In [25]:
m.components                        #shows components of the model

   # |      Attribute Name |      Component Name |      Component Type
---- | ------------------- | ------------------- | -------------------
   0 |         Groundstate |         Groundstate |          Expression
   1 | First_Excited_State | First Excited State |          Expression
   2 | Second_excited_St.. | Second excited St.. |          Expression
   3 |              Offset |              Offset |              Offset

Sets first values for Fit

In [38]:
g1.x0.value = 1153   # Guess for centre wavelength
g1.x0.bmax = 1160     # Max value for centre wavelength
g1.x0.bmin = 1120     # Min value for centre wavelength

g1.fwhm.value = 50      #Guess for FWHM
g1.fwhm.bmax = 100       #Maxvalue for FWHM
g1.fwhm.bmin = 25       #Minvalue for FWHM

g1.height.value = 500       #Guess for peak Intensity
g1.height.bmax = 4000      #Maxvalue for peak Intesity
g1.height.bmin = 1         #Minvalue for peak Intensity

g2.x0.value = 1080   # Guess for centre wavelength
g2.x0.bmax = 1090     # Max value for centre wavelength
g2.x0.bmin = 1050     # Min value for centre wavelength

g2.fwhm.value = 50      #Guess for FWHM
g2.fwhm.bmax = 75       #Maxvalue for FWHM
g2.fwhm.bmin = 25       #Minvalue for FWHM

g2.height.value = 500       #Guess for peak Intensity
g2.height.bmax = 1500      #Maxvalue for peak Intesity
g2.height.bmin = 1         #Minvalue for peak Intensity

g3.x0.value = 1030  # Guess for centre wavelength
g3.x0.bmax = 1040     # Max value for centre wavelength
g3.x0.bmin = 1015     # Min value for centre wavelength

g3.fwhm.value = 50      #Guess for FWHM
g3.fwhm.bmax = 75       #Maxvalue for FWHM
g3.fwhm.bmin = 25       #Minvalue for FWHM

g3.height.value = 500       #Guess for peak Intensity
g3.height.bmax = 1500      #Maxvalue for peak Intesity
g3.height.bmin = 1         #Minvalue for peak Intensity


bkg.offset.value = 5  #Background to be substracted

In [39]:
s.plot()                #plots map
m.plot(plot_components=True)              # adds fit model to map
#m.gui()

In [40]:
m.fit(bounded=True)         #fits function in selected point to measured curve
m.print_current_values()    #Prints values of fit in the set pixel

Parameter Name,Free,Value,Std,Min,Max
fwhm,True,38.0112,0.269118,25,100
height,True,2435.04,13.1647,1,4000
x0,True,1153.88,0.096883,1120,1160
Parameter Name,Free,Value,Std,Min,Max
fwhm,True,40.292,0.613254,25,75
height,True,1304.43,13.9475,1,1500
x0,True,1088.09,0.214471,1050,1090
Parameter Name,Free,Value,Std,Min,Max
fwhm,True,34.2249,1.59749,25,75
height,True,429.304,13.6834,1,1500


In [27]:
m.plot()

Traceback (most recent call last):
  File "C:\Users\Gunnar\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 224, in process
    func(*args, **kwargs)
  File "C:\Users\Gunnar\anaconda3\lib\site-packages\hyperspy\drawing\image.py", line 615, in on_key_press
    self.gui_adjust_contrast()
  File "C:\Users\Gunnar\anaconda3\lib\site-packages\hyperspy\drawing\image.py", line 586, in gui_adjust_contrast
    ceditor = ImageContrastEditor(self)
  File "C:\Users\Gunnar\anaconda3\lib\site-packages\hyperspy\signal_tools.py", line 795, in __init__
    self.image._vmin_percentile.split('th')[0])
AttributeError: 'float' object has no attribute 'split'


In [ ]:
m.multifit(bounded=True, iterpath='serpentine')    #fits to the whole map

Plots Centre Wavelength of Fit g1

In [55]:
g1.x0.plot(colorbar=True,                             
                             cmap='Spectral_r',
                             centre_colormap = False,
                             saturated_pixels=0)

g2.x0.plot(colorbar=True,                             
                             cmap='Spectral_r',
                             centre_colormap = False,
                             saturated_pixels=0)

g3.x0.plot(colorbar=True,                             
                             cmap='Spectral_r',
                             centre_colormap = False,
                             saturated_pixels=0)


Added statistical Analysis written by Giorgio Divitini

In [ ]:
wavelength = g1.x0.as_signal()
wavelength.data.mean()
wavelength.data.std()

Plots peak intensity of Fit g1

In [57]:
g1.height.plot(colorbar=True,                             
                             cmap='viridis',
                             centre_colormap = False,
                             saturated_pixels=0)

g2.height.plot(colorbar=True,                             
                             cmap='viridis',
                             centre_colormap = False,
                             saturated_pixels=0)

g3.height.plot(colorbar=True,                             
                             cmap='viridis',
                             centre_colormap = False,
                             saturated_pixels=0)


Plots FWHM of Fit g1

In [29]:
g1.fwhm.plot(colorbar=True,                             
                             cmap='magma',
                             centre_colormap = False,
                             saturated_pixels=0)

g2.fwhm.plot(colorbar=True,                             
                             cmap='magma',
                             centre_colormap = False,
                             saturated_pixels=0)

g3.fwhm.plot(colorbar=True,                             
                             cmap='magma',
                             centre_colormap = False,
                             saturated_pixels=0)


Extract and save a spectrum from a pixel

In [ ]:
s.plot() #Plot Map, move cursor to desired pixel

In [ ]:
Px , Py = s.axes_manager.indices  #reads in cursor positions
yvals = s.inav[Px, Py]            # gets y-axis at cursor position
xvals = dx.offset+np.arange(len(yvals.data))*(dx.scale*binning) #creates a wavelength-axis to save

Saves the point as hdf5 (Origin readable)

In [ ]:
hf = h5py.File('C:\\Filepath\\Filename.h5', 'w') #replace w with r+ if saving multiple points into the same file
h1 = hf.create_group('Point1')
h1.create_dataset('Wavelength-Axis', data=xvals)
h1.create_dataset('Intensity', data= yvals)
hf.close()